# 모듈 불러오기

#### 기본

In [ ]:
import numpy as np
import pandas as pd
import joblib

In [ ]:
# !conda install numpy 
# !conda install pandas
# !conda install scikit-learn
# !conda install scipy
# !conda install tensorflow
# !conda install matplotlib
# !conda install seaborn

# !pip install xgboost
# !pip install lightgbm
# !pip install catboost
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !pip install hyperopt
# !pip install -U imbalanced-learn
# !pip install missingno

#### 전처리

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

from sklearn import impute
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer

#### 리샘플링

In [ ]:
from imblearn.over_sampling import (
    RandomOverSampler, 
    ADASYN, 
    SMOTE
)
from imblearn.under_sampling import (
    RandomUnderSampler, 
    TomekLinks, 
    CondensedNearestNeighbour, 
    OneSidedSelection, 
    EditedNearestNeighbours, 
    NeighbourhoodCleaningRule
)

#### 분석

In [ ]:
from scipy.stats import skew, kurtosis
from scipy.stats import ttest_ind, f_oneway, pearsonr, chi2_gen

#### 회귀

In [ ]:
from sklearn.linear_model import LinearRegression as RL
from sklearn.neighbors import KNeighborsRegressor as KNNR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR
from xgboost import XGBRegressor as XGBR
from lightgbm import LGBMRegressor as LGBMR
from catboost import CatBoostRegressor as CBR

from lightgbm import plot_importance as lgbm_plot_importance
from xgboost import plot_importance as xgb_plot_importance
from catboost import Pool

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

#### 분류

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from xgboost import XGBClassifier as XGBC
from lightgbm import LGBMClassifier as LGBMC
from catboost import CatBoostClassifier as CBC

from sklearn.metrics import confusion_matrix as cmatrix
from sklearn.metrics import classification_report as creport
from sklearn.metrics import recall_score as recall
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

#### 교차검증

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import cv
import hyperopt

from sklearn.model_selection import (
    StratifiedKFold, # 분류
    KFold, # 회귀
    # GroupKFold, 
    # RepeatedKFold, 
    # StratifiedGroupKFold, 
    # RepeatedStratifiedKFold
)

#### 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')  

import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

# 데이터 로드

In [ ]:
folder_name = '1th'

x_train = pd.read_csv(f'./{folder_name}/x_train.csv', sep=',', encoding='utf-8')
y_train = pd.read_csv(f'./{folder_name}/y_train.csv', sep=',', encoding='utf-8')
x_test = pd.read_csv(f'./{folder_name}/x_test.csv', sep=',', encoding='utf-8')
y_test = pd.read_csv(f'./{folder_name}/y_test.csv', sep=',', encoding='utf-8')
x_final = pd.read_csv(f'./{folder_name}/test_x.csv', sep=',', encoding='utf-8')

# 회귀 모델링 선택

In [ ]:
models =    {
                'RL': RL(n_jobs=-1),
                'KNNR': KNNR(n_neighbors=5),
                'DTR': DTR(max_depth=5, random_state=1),
                'SVR': SVR(kernel='rbf', C=1),
                'RFR': RFR(max_depth=5, random_state=1),
                'LGBMR': LGBMR(max_depth=5, num_leaves=31, learning_rate=0.1),
                'CBR': CBR(iterations=100, cat_features=np.where(x_train.dtypes != np.float)[0], random_state=1)
            }

results = {}
r2_scores = {}

for key in models:
    model = models[key]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    mse_result = mse(y_test, y_pred)
    r2_result = r2_score(y_test, y_pred)
    r2_scores[key] = r2_result
    
    print('=' * 20, key, '=' * 20)
    print('mse : ', mse_result)
    print('r2_score : ', r2_result)
    
    results[key] = model

In [ ]:
for key in results:
    results[key]
    plt.figure(figsize=(16, 6))
    plt.barh(y=list(x_train), width=results[key].feature_importances_)
    plt.show()

In [ ]:
# 시각화
plt.figure(figsize=(16, 6))
plt.barh(y=list(r2_scores), width=r2_scores.values())
plt.xlabel('R2_Score')
plt.ylabel('Model')
plt.show()

# 분류 모델링 선택

In [ ]:
models = {
        'LR': LR(C=1, random_state=1, n_jobs=-1),
        'KNN': KNNC(n_neighbors=5, n_jobs=-1),
        'DTC': DTC(max_depth=5, random_state=1),
        'SVC': SVC(kernel='rbf', C=1),
        'RFC': RFC(max_depth=5, random_state=1),
        'LGBMC': LGBMC(max_depth=5, num_leaves=31, learning_rate=0.1, random_state=1),
        'CBC': CBC(iterations=100, cat_features=np.where(x_train.dtypes != np.float)[0], random_state=1)
        }

results = {}
acc_results = {}
recall_results = {}
y_preds = {}

for key in models:
    model = models[key]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    results[key] = model
    y_preds[key] = y_pred
    acc_results[key] = acc(y_test, y_pred)
    recall_results[key] = recall(y_test, y_pred)

In [ ]:
key = 'CBC'

print('=' * 20, key, '=' * 20)
print('confusion_matrix\n', cmatrix(y_test, y_preds[key]))
print('classification_report\n', creport(y_test, y_preds[key]))
print('=' * 60)

# 시각화

#### 트리 모델 시각화

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize = (20, 8))
plot_tree(model, feature_names = x_train.columns,
          filled = True, fontsize = 10)

#### 모델별 정확도 시각화

In [ ]:
tmp = pd.Series(acc_results)

plt.figure(figsize=(16, 6))
plt.title('accuracy')
plt.barh(tmp.index, tmp.values)
plt.show()

#### 모델별 재현율 시각화

In [ ]:
tmp = pd.Series(recall_results)

plt.figure(figsize=(16, 6))
plt.title('recall')
plt.barh(tmp.index, tmp.values)
plt.show()

#### 모델별 학습성능, 평가성능 출력

In [ ]:
print('모델 : 학습성능 / 평가성능 (Accuracy)')
for key in results:
    print(f'{key:10} : {round(results[key].score(x_train, y_train), 4):6} / {round(results[key].score(x_test, y_test), 4):6}\n')

#### 변수 중요도 시각화

In [ ]:
key = 'LGBMC'
tmp = pd.Series(data=results[key].feature_importances_, index=list(x_train)).sort_values()

plt.figure(figsize=(16, 6))
plt.barh(y=tmp.index, width=tmp.values)
plt.show()

key = 'CBC'
tmp = pd.Series(data=results[key].feature_importances_, index=list(x_train)).sort_values()

plt.figure(figsize=(16, 6))
plt.barh(y=tmp.index, width=tmp.values)
plt.show()

In [ ]:
# feature importance 시각화 함수
def plot_feature_importance(importance, names, topn = 'all'):
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    plt.figure(figsize=(10,8))
    sns.barplot(x='feature_importance', y='feature_names', data = fi_df)

    plt.xlabel('importance')
    plt.ylabel('feature names')
    plt.grid()

    return fi_df

plot_feature_importance(results[key].feature_importances_, list(x_train))

# AI 모델링 선택 cross_val_score()

#### KFold 회귀

In [ ]:
models =    {
            'RL': RL(n_jobs=-1),
            'KNNR': KNNR(n_neighbors=5),
            'DTR': DTR(max_depth=5, random_state=1),
            'SVR': SVR(kernel='rbf', C=1),
            'RFR': RFR(max_depth=5, random_state=1),
            'XGBR': XGBR(max_depth=5, num_leaves=31, learning_rate=0.1),
            'LGBMR': LGBMR(max_depth=5, num_leaves=31, learning_rate=0.1),
            'CBR': CBR(iterations=100, cat_features=np.where(x_train.dtypes != np.float)[0], random_state=1),
            }

results = {}

for key in models:
    cv_score = cross_val_score(
                                models[key], 
                                x_train, 
                                y_train, 
                                cv=KFold(n_splits=5)
                                ) 

    print('=' * 20, key, '=' * 20)
    print(cv_score)
    print('평균:', cv_score.mean())
    print('표준편차:', cv_score.std())
    
    results[key] = cv_score.mean()

In [ ]:
# 시각화
plt.figure(figsize=(5, 5))
plt.barh(y=list(results), width=results.values())
plt.xlabel('Score')
plt.ylabel('Model')
plt.show()

#### SatisfiedFold 분류

In [ ]:
models =    {
            # 'LR': LR(C=1, random_state=1, n_jobs=-1),
            # 'KNN': KNNC(n_neighbors=5),
            # 'DT': DTC(max_depth=5, random_state=1),
            # 'SVC': SVC(kernel='rbf', C=1, random_state=1),
            'RFC': RFC(max_depth=5, random_state=1, n_jobs=-1),
            'XGBC': XGBC(max_depth=5, learning_rate=0.1),
            'LGBMC': LGBMC(max_depth=5, num_leaves=31, learning_rate=0.1),
            'CBC': CBC(iterations=100, cat_features=np.where(x_train.dtypes != np.float)[0], random_state=1, verbose=False),
            }

results = {}

for key in models:
    cv_score = cross_val_score(
                                models[key], 
                                x_train, 
                                y_train, 
                                cv=StratifiedKFold(n_splits=5),
                                verbose=False
                                ) 

    print('=' * 20, key, '=' * 20)
    print(cv_score)
    print('평균:', cv_score.mean())
    print('표준편차:', cv_score.std())
    
    results[key] = cv_score.mean()

In [ ]:
# 시각화
plt.figure(figsize=(16, 6))
plt.barh(y=list(results), width=results.values())
plt.xlabel('Score')
plt.ylabel('Model')
plt.show()